# 1. Initialization

### i) Configure RPCs & deployer

In [ ]:
from dotenv import load_dotenv
import os
import boa
from eth_account import Account
from web3 import Web3
import logging
import subprocess
import time

from ABIs import createX_abi

# logging.basicConfig(filename="deploy.log", filemode='a', level=logging.INFO, format="%(asctime)s -  %(levelname)s - %(message)s")
logging.basicConfig(level=logging.INFO, format="%(asctime)s -  %(levelname)s - %(message)s")

load_dotenv()
main_chain = "sepolia"

PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")

RPCs = {
    "sepolia": "https://eth-sepolia.public.blastapi.io",
    "base-sepolia": "https://sepolia.base.org",
    "optimism-sepolia": "https://sepolia.optimism.io",
    "arbitrum-sepolia": "https://sepolia-rollup.arbitrum.io/rpc",
}

createX_address = "0xba5Ed099633D3B313e4D5F7bdc1305d3c28ba5Ed"

state_dict = {}

deployer = Account.from_key(PRIVATE_KEY)

for key in RPCs.keys():
    state_dict[key] = {}
    state_dict[key]["rpc"] = RPCs[key]
    boa.set_network_env(RPCs[key])
    boa.env.add_account(deployer)
    state_dict[key]["boa"] = boa.env  # oops
    state_dict[key]["w3"] = Web3(Web3.HTTPProvider(RPCs[key]))
    state_dict[key]["createx"] = boa.loads_abi(createX_abi).at(createX_address)

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        logging.info(f"Working with {boa.env.eoa} on {key}, id {boa.env.evm.patch.chain_id}")
        logging.info(
            f"Chain balance is {state_dict[key]['w3'].eth.get_balance(boa.env.eoa)/1e18 :.3f} ETH"
        )
        logging.info(f"CreateX test: {state_dict[key]['createx'].computeCreate2Address(b'',b'')}")

### ii) Parse LZ deployments data (libs and endpoints)

In [ ]:
from LZDeployments import LZDeployments

lz = LZDeployments()
for chain in state_dict:
    metadata = lz.get_chain_metadata(chain)["metadata"]
    dvn_data = lz.get_chain_metadata(chain)

    # Update state dict
    state_dict[chain].update(
        {
            "eid": metadata["eid"],
            "endpoint": metadata["endpointV2"],
            "send_lib": metadata.get("sendUln302", "unavailable"),
            "receive_lib": metadata.get("receiveUln302", "unavailable"),
            "read_lib": metadata.get("readLib1002", "unavailable"),
            "dvns": dvn_data["dvns"],
            "executor": metadata.get("executor", "0x0000000000000000000000000000000000000000"),
        }
    )

    # Print info
    logging.info(f"LZ details for {chain}:")
    logging.info(f"Chain eID: {metadata['eid']}\nEndpoint address: {metadata['endpointV2']}")
    logging.info(f"DVNs: {len(dvn_data['dvns'])}, Read DVNs: {len(dvn_data['dvns_lzread'])}")
    logging.info(
        f"Send lib: {state_dict[chain]['send_lib']}\n"
        f"Receive lib: {state_dict[chain]['receive_lib']}\n"
        f"Read lib: {state_dict[chain]['read_lib']}\n"
        f"Executor: {state_dict[chain]['executor']}\n---"
    )

# 2. Contracts deployment

### i) Deploy main view contract and block oracles 

In [ ]:
# On mainnet viewer will tell us ground truth block data
# On other chains block oracles are consensus of various messengers (LZ in this script)
guard_bytes = bytes.fromhex(boa.env.eoa[2:] + "00")
bytes_view = os.urandom(11)
bytes_oracle = os.urandom(11)

salt_view = guard_bytes + bytes_view
salt_oracle = guard_bytes + bytes_oracle

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        if key == main_chain:
            # deploying view contract on mainnet
            contract_deployer = boa.load_partial("../contracts/MainnetBlockView.vy")
            deploycode = contract_deployer.compiler_data.bytecode
            address = state_dict[key]["createx"].deployCreate2(salt_view, deploycode)
            # contract = contract_deployer(address)
            logging.info(f"Block view deployed at {address} on {key}")
            state_dict[key]["oracle"] = contract_deployer.at(address)
        else:
            # deploying block oracle on other chains
            contract_deployer = boa.load_partial("../contracts/BlockOracle.vy")
            bytecode = contract_deployer.compiler_data.bytecode
            args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
            deploycode = bytecode + args

            address = state_dict[key]["createx"].deployCreate2(salt_oracle, deploycode)
            # contract = contract_deployer(1)  # commit threshold
            logging.info(f"Block oracle deployed at {address} on {key}")
            state_dict[key]["oracle"] = contract_deployer.at(address)

In [ ]:
# Deploy LZBlockRelay on each chain
# This contract is used to send and receive messages via LZ, including LZRead and chained broadcasting
guard_bytes = bytes.fromhex(boa.env.eoa[2:] + "00")
bytes_relay = os.urandom(11)

salt_relay = guard_bytes + bytes_relay
# salt_relay = bytes.fromhex('73241e98090042a718f7eb1af07fad27ff09a3f3002b281cb382b00f03a22004')
for key in state_dict:
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        contract_deployer = boa.load_partial("../contracts/messengers/LZBlockRelay.vy")
        bytecode = contract_deployer.compiler_data.bytecode
        args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
        deploycode = bytecode + args
        address = state_dict[key]["createx"].deployCreate2(salt_relay, deploycode)
        contract = contract_deployer.at(address)
        print(f"LZ Messenger deployed at {contract.address} on {key}")
        state_dict[key]["block_relay"] = contract

# 3. Contracts configuration

## i) Oracles configuration

#### Initialize with all deployed peer ids and read libs

In [ ]:
# Prepare initialization parameters for block relay contracts
for key in state_dict.keys():
    if key == main_chain:
        continue

    # Basic parameters
    endpoint = state_dict[key]["endpoint"]
    default_gas_limit = 300_000
    read_channel = 4294967295 if state_dict[key]["read_lib"] != "unavailable" else 0

    # Use dict to manage unique peer relationships
    peers_dict = {}

    # 1. If read-enabled:
    # - add self as read peer
    # - add all other chains as peers for broadcasting
    if state_dict[key]["read_lib"] != "unavailable":
        peers_dict[read_channel] = state_dict[key]["block_relay"].address

        for target_key in state_dict.keys():
            if target_key != main_chain and target_key != key:
                peers_dict[state_dict[target_key]["eid"]] = state_dict[target_key][
                    "block_relay"
                ].address

    # 2. For all chains: add read-enabled chains as peers (to receive their broadcasts)
    # duplicates are managed by dict structure
    for source_key in state_dict.keys():
        if (
            source_key != main_chain
            and source_key != key
            and state_dict[source_key]["read_lib"] != "unavailable"
        ):
            peers_dict[state_dict[source_key]["eid"]] = state_dict[source_key][
                "block_relay"
            ].address

    # Convert dict to lists
    peer_eids = list(peers_dict.keys())
    peers = list(peers_dict.values())

    # Prepare lib configuration
    oapps = []
    channels = []
    libs = []
    lib_types = []

    # Add send and receive libs
    # iterate over peers_dict
    for peer_eid, peer_address in peers_dict.items():
        if peer_eid == read_channel or peer_eid == state_dict[key]["eid"]:
            # skip read channel and own eid
            continue
        if state_dict[key]["send_lib"] != "unavailable":
            oapps.append(peer_address)
            channels.append(peer_eid)
            libs.append(state_dict[key]["send_lib"])
            lib_types.append(1)

        if state_dict[key]["receive_lib"] != "unavailable":
            oapps.append(peer_address)
            channels.append(peer_eid)
            libs.append(state_dict[key]["receive_lib"])
            lib_types.append(2)

    # now add read libs explicitly
    if state_dict[key]["read_lib"] != "unavailable":
        oapps.extend([state_dict[key]["block_relay"].address] * 2)
        channels.extend([read_channel, read_channel])
        libs.extend([state_dict[key]["read_lib"], state_dict[key]["read_lib"]])
        lib_types.extend([1, 2])

    logging.info(f"init parameters on {key}")
    logging.info(f"Peer_eids: {peer_eids}")
    logging.info(f"Peers: {peers}")
    logging.info(f"Oapps: {oapps}")
    logging.info(f"Channels: {channels}")
    logging.info(f"Libs: {libs}")
    logging.info(f"Lib types: {lib_types}")
    logging.info(f"Initializing block relay on {key}...")

    # Initialize
    relay_contract = state_dict[key]["block_relay"]
    if not relay_contract.is_initialized():
        relay_contract.initialize(
            endpoint,
            default_gas_limit,
            read_channel,
            peer_eids,
            peers,
            oapps,
            channels,
            libs,
            lib_types,
        )
    else:
        logging.info(f"Relay on {key} already initialized")

#### Add block oracles and committer

In [ ]:
for key in state_dict.keys():
    if key == main_chain:
        continue
    relay_contract = state_dict[key]["block_relay"]

    # add block oracles to every relayer
    if relay_contract.block_oracle() == state_dict[key]["oracle"].address:
        logging.info(f"Skipping {key} - already set")
    else:
        relay_contract.set_block_oracle(state_dict[key]["oracle"].address)
        logging.info(f"Set block oracle tx on {key}")

    # now add relayer as committer to block oracle
    oracle_contract = state_dict[key]["oracle"]
    if oracle_contract.is_committer(relay_contract.address):
        logging.info(f"Skipping {key} - already a committer")
    else:
        oracle_contract.add_committer(relay_contract.address, True)
        logging.info(f"Add committer tx on {key}")

    # add mainnet view contract on read chains (as read source)
    if state_dict[key]["read_lib"] != "unavailable":
        contract = state_dict[key]["block_relay"]
        contract.set_read_config(
            True, state_dict[main_chain]["eid"], state_dict[main_chain]["oracle"].address
        )
        logging.info(f"Set read config on {key}")

## ii) LZ Configuration

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
def get_vyper_abi(filepath):
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


ABI_RELAY = get_vyper_abi("../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../contracts/BlockOracle.vy")
for key in state_dict.keys():
    if key == main_chain:
        continue
    state_dict[key]["block_relay_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["block_relay"].address, abi=ABI_RELAY
    )
    state_dict[key]["oracle_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["oracle"].address, abi=ABI_ORACLE
    )
account = Web3().eth.account.from_key(deployer.key)


def send_tx_single(w3, func, acc, value=0, gas_multiplier=2):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    tx["gas"] = int(gas_multiplier * w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0):
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value)
            success = True
        except Exception as e:
            if "replacement transaction underpriced" in str(e) or "nonce too low" in str(e):
                print(str(e), "Retrying...")
                success = False
                time.sleep(0.5)
            else:
                raise e
    return tx_hash

### Simple reads without broadcast

In [ ]:
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"Current block number on {main_chain}: {main_block}")
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        logging.info(f"Skipping {key}")
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]

    # First quote read fee
    fee = contract_w3.functions.quote_read_fee().call()
    logging.info(f"Read fee: {fee} on {key}")

    # Then request read
    func = contract_w3.functions.request_block_hash([], [])
    tx_hash = send_tx(state_dict[key]["w3"], func, account, int(1.2 * fee))
    logging.info(f"Tx: {tx_hash.hex()} on {key}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    try:
        logging.info(f"Number difference: {main_block - number}")
    except Exception:
        pass

### Now separate broadcast

In [ ]:
broadcaster = "base-sepolia"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
receive_eids = [
    state_dict[key]["eid"] for key in state_dict.keys() if key not in [broadcaster, main_chain]
]
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
broadcast_fees = [10 * fee for fee in broadcast_fees]
logging.info(f"LZSend fees: {broadcast_fees} on {broadcaster}")

# broadcast call
func = broadcaster_w3.functions.broadcast_latest_block(receive_eids, broadcast_fees)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, sum(broadcast_fees))
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

### Now reads with broadcast

In [ ]:
# we can quote read fee now
# let's pick one of the read chains and quote fee
broadcaster = "base-sepolia"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
receive_eids = [
    state_dict[key]["eid"] for key in state_dict.keys() if key not in [broadcaster, main_chain]
]
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
broadcast_fees = [int(fee * 1.2) for fee in broadcast_fees]
logging.info(f"LZSend fees: {broadcast_fees} on {broadcaster}")

BROADCAST_GAS = 2_000_000
read_fee_with_broadcast = broadcaster_w3.functions.quote_read_fee(
    0, BROADCAST_GAS, sum(broadcast_fees)
).call()
logging.info(f"LZRead fee with broadcast: {read_fee_with_broadcast} on {broadcaster}")

# magic broadcast call
main_block = state_dict[main_chain]["w3"].eth.block_number
func = broadcaster_w3.functions.request_block_hash(receive_eids, broadcast_fees, 0, BROADCAST_GAS)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, read_fee_with_broadcast)
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    try:
        logging.info(f"Number difference: {main_block - number}")
    except Exception:
        pass

### Request remote lzread

In [ ]:
# Pick a read-supporting chain to request the read from
read_chain = "arbitrum-sepolia"
requesting_chain = "optimism-sepolia"

# first use read contracts to estimate read and broadcast fees
fee_multiplier = 2
read_contract = state_dict[read_chain]["block_relay_w3"]
read_fee = fee_multiplier * read_contract.functions.quote_read_fee().call()
broadcast_fee = (
    fee_multiplier
    * read_contract.functions.quote_broadcast_fees([state_dict[requesting_chain]["eid"]]).call()[0]
)
logging.info(f"Read fee: {read_fee} on {read_chain}")
logging.info(f"Broadcast fee: {broadcast_fee} on {read_chain}")

# now request remote read
requesting_contract = state_dict[requesting_chain]["block_relay_w3"]
func = requesting_contract.functions.request_remote_read(
    state_dict[read_chain]["eid"], read_fee, broadcast_fee, 500_000
)
msg_value = 10 * (read_fee + broadcast_fee)
tx_hash = send_tx(state_dict[requesting_chain]["w3"], func, account, msg_value)
logging.info(f"Tx: {tx_hash.hex()} on {requesting_chain}")

### Submit block header

In [ ]:
from utils import encode_headers

chain_key = "optimism-sepolia"

# boa because we have it initialized
block_number = state_dict[chain_key]["oracle"].last_confirmed_block_number()
print(f"Last confirmed block on {chain_key}: {block_number}")
block_hash = state_dict[chain_key]["oracle"].block_hash(block_number)
print(f"Block hash: {block_hash.hex()}")
# block_data = state_dict[chain_key]["oracle"].block_data(block_number)
hash1, _, root1, number1, timestamp1 = state_dict[chain_key]["oracle"].block_header(block_number)
print(f"Block headers: {hash1.hex()}, {root1.hex()}, {number1}, {timestamp1}")
block_data = state_dict[main_chain]["w3"].eth.get_block(block_number, full_transactions=False)
encoded_headers = encode_headers(block_data)
print(f"Encoded headers: {encoded_headers.hex()}")
state_dict[chain_key]["oracle"].submit_block_header(encoded_headers)
# print(block_data)
headers = state_dict[chain_key]["oracle"].block_header(block_number)
print(
    f"Block headers: {headers[0].hex()}, {headers[1].hex()}, {headers[2].hex()}, {headers[3]}, {headers[4]}"
)

## Manage DVNs

### Get current DVN config


In [ ]:
from ABIs import lzreadlib_abi
from eth_abi import decode


def decode_dvn_config(hex_data: str, config_type="read", executor=False):
    hex_data = hex_data.replace("0x", "")

    types = (
        ["(uint64,uint8,uint8,uint8,address[],address[])"]
        if config_type != "read"
        else ["(address,uint8,uint8,uint8,address[],address[])"]
    )
    if executor:
        types = ["(uint32,address)"]
    decoded = decode(types, bytes.fromhex(hex_data))
    if executor:
        return {"executor": decoded[0][1]}
    return {
        "confirmations" if config_type != "read" else "executor": decoded[0][0],
        "requiredDVNCount": decoded[0][1],
        "optionalDVNCount": decoded[0][2],
        "optionalDVNThreshold": decoded[0][3],
        "requiredDVNs": [Web3.to_checksum_address(addr) for addr in decoded[0][4]],
        "optionalDVNs": [Web3.to_checksum_address(addr) for addr in decoded[0][5]],
    }


# # Usage:
# hex_data = "0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000d680ec569f269aa7015f7979b4f1239b5aa4582c0000000000000000000000000000000000000000000000000000000000000000"
# print(decode_dvn_config(hex_data))
# prepare reverse dict eid -> name
eid_to_name = {state_dict[key]["eid"]: key for key in state_dict.keys()}

CONFIG_TYPE_READ = 1
CONFIG_TYPE_ULN = 2
CONFIG_TYPE_EXECUTOR = 1
# 1. For each chain, for each lib, for each peer - check DVN config
for key in state_dict.keys():
    logging.info(f"Checking DVN config on {key}...")
    if key == main_chain:
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]
    logging.info(f"Relayer contract: {contract_w3.address}")
    configured_eids = contract_w3.functions.get_configured_eids().call()
    logging.info(f"Configured EIDs: {configured_eids}")
    state_dict[key]["current_dvn_config"] = {}
    w3 = state_dict[key]["w3"]
    lib_send = (
        w3.eth.contract(address=state_dict[key]["send_lib"], abi=lzreadlib_abi)
        if state_dict[key]["send_lib"] != "unavailable"
        else None
    )
    lib_receive = (
        w3.eth.contract(address=state_dict[key]["receive_lib"], abi=lzreadlib_abi)
        if state_dict[key]["receive_lib"] != "unavailable"
        else None
    )
    lib_read = (
        w3.eth.contract(address=state_dict[key]["read_lib"], abi=lzreadlib_abi)
        if state_dict[key]["read_lib"] != "unavailable"
        else None
    )
    for eid in configured_eids:
        if eid > 4294965694:
            eid_to_name[eid] = "read_channel"
        logging.info(f"EID: {eid} -> {eid_to_name[eid]}")
        oapp = contract_w3.functions.LZ_PEERS(eid).call()
        state_dict[key]["current_dvn_config"][eid] = {}
        logging.info(f"Peer oapp: {oapp}")
        if lib_send:
            try:
                send_config = lib_send.functions.getConfig(eid, oapp, CONFIG_TYPE_ULN).call()
                send_config = decode_dvn_config(send_config.hex(), "send")
                executor_config = lib_send.functions.getConfig(
                    eid, oapp, CONFIG_TYPE_EXECUTOR
                ).call()
                executor_config = decode_dvn_config(executor_config.hex(), "send", True)
                send_config["executor"] = executor_config["executor"]
            except Exception as e:
                send_config = None
                logging.debug(f"Send lib config: ERR on {key} - {e}")
        if lib_receive:
            try:
                receive_config = lib_receive.functions.getConfig(eid, oapp, CONFIG_TYPE_ULN).call()
                receive_config = decode_dvn_config(receive_config.hex(), "receive")
            except Exception as e:
                receive_config = None
                logging.debug(f"Receive lib config: ERR on {key} - {e}")
        if lib_read:
            try:
                read_config = lib_read.functions.getConfig(eid, oapp, CONFIG_TYPE_READ).call()
                read_config = decode_dvn_config(read_config.hex(), "read")
            except Exception as e:
                read_config = None
                logging.debug(f"Read lib config: ERR on {key} - {e}")
        logging.info(f"Send lib config: {send_config}")
        logging.info(f"Receive lib config: {receive_config}")
        logging.info(f"Read lib config: {read_config}")
        state_dict[key]["current_dvn_config"][eid]["send"] = send_config
        state_dict[key]["current_dvn_config"][eid]["receive"] = receive_config
        state_dict[key]["current_dvn_config"][eid]["read"] = read_config
    logging.info("-" * 100)

### Plan desired DVN config

In [ ]:
def inject_curve_dvns(dvns_list, chain_key):
    # Curve DVN addresses per chain
    curve_dvns = {
        "sepolia": "0x3dc328b85d66362d9b5b359789b5bf3997dbf12e",
        "base-sepolia": "0x6310fb9f8efca6b486276c3ba74a4dd508e41e7a",
        "optimism-sepolia": "0x48f830f47d6c98ae323acd3c6cd1cbac8c6f3ce5",
        "arbitrum-sepolia": "0x2076f14b292abdbef7e2871ca93608755fa741a9",
    }

    # Skip if chain not in mapping
    if chain_key not in curve_dvns:
        return dvns_list

    # Create Curve DVN entry
    curve_dvn = {
        "address": curve_dvns[chain_key],
        "version": 2,
        "canonicalName": "Curve Finance",
        "id": "curve",
        "lzReadCompatible": True,
    }

    # Add to list if not already present
    if not any(d["address"].lower() == curve_dvn["address"].lower() for d in dvns_list):
        dvns_list.append(curve_dvn)

    return dvns_list


# Build desired DVN configurations
for key in state_dict.keys():
    if key == main_chain:
        continue

    logging.info(f"\nAnalyzing {key}...")
    contract_w3 = state_dict[key]["block_relay_w3"]
    configured_eids = contract_w3.functions.get_configured_eids().call()
    # Initialize if not exists
    if "desired_dvn_config" not in state_dict[key]:
        state_dict[key]["desired_dvn_config"] = {}

    # Get all DVNs including Curve for this chain
    dvns = inject_curve_dvns(state_dict[key]["dvns"], key)

    # For each peer relationship (only process sends, receives will be populated)
    for eid in configured_eids:
        if eid > 4294965694:  # Read channel
            if state_dict[key]["read_lib"] != "unavailable":
                # Get read-compatible DVNs
                read_dvns = [
                    dvn
                    for dvn in dvns
                    if dvn.get("lzReadCompatible", False)
                    and dvn["id"] in ["layerzero-labs", "nethermind"]
                ]
                if read_dvns:
                    state_dict[key]["desired_dvn_config"][eid] = {
                        "send": {
                            "requiredDVNs": [dvn["address"] for dvn in read_dvns],
                            "optionalDVNs": [],
                            "executor": state_dict[key]["executor"],
                        },
                        "receive": {"requiredDVNs": [], "optionalDVNs": []},
                    }
            continue

        peer_chain = eid_to_name.get(eid)
        if not peer_chain:
            continue

        logging.info(f"\n{key} -> {peer_chain} relationship:")
        peer_dvns = inject_curve_dvns(state_dict[peer_chain]["dvns"], peer_chain)

        # Get DVNs by ID for both chains
        # Only consider non-READ-compatible DVNs for ULN
        my_dvns_by_id = {
            d["id"]: d["address"]
            for d in dvns
            if not d.get("lzReadCompatible", False) or d["id"] == "curve"
        }
        peer_dvns_by_id = {
            d["id"]: d["address"]
            for d in peer_dvns
            if not d.get("lzReadCompatible", False) or d["id"] == "curve"
        }

        # Check available required DVNs
        required_ids = []

        # Try to get all three preferred DVNs
        required_ids = []
        for dvn_id in ["layerzero-labs", "nethermind"]:
            if dvn_id in my_dvns_by_id and dvn_id in peer_dvns_by_id:
                required_ids.append(dvn_id)
        if not required_ids:
            logging.warning(f"No matching DVNs found for {key}->{peer_chain}")
            continue

        optional_ids = []
        for dvn_id in ["curve"]:
            if dvn_id in my_dvns_by_id and dvn_id in peer_dvns_by_id:
                optional_ids.append(dvn_id)

        # Initialize configs if not exist
        if eid not in state_dict[key]["desired_dvn_config"]:
            state_dict[key]["desired_dvn_config"][eid] = {
                "send": {"requiredDVNs": [], "optionalDVNs": []},
                "receive": {"requiredDVNs": [], "optionalDVNs": []},
            }

        if "desired_dvn_config" not in state_dict[peer_chain]:
            state_dict[peer_chain]["desired_dvn_config"] = {}

        if state_dict[key]["eid"] not in state_dict[peer_chain].get("desired_dvn_config", {}):
            state_dict[peer_chain]["desired_dvn_config"][state_dict[key]["eid"]] = {
                "send": {"requiredDVNs": [], "optionalDVNs": []},
                "receive": {"requiredDVNs": [], "optionalDVNs": []},
            }

        # Set send config for this chain
        state_dict[key]["desired_dvn_config"][eid]["send"] = {
            "requiredDVNs": [my_dvns_by_id[id] for id in required_ids],
            "optionalDVNs": [my_dvns_by_id[id] for id in optional_ids],
            "executor": state_dict[key]["executor"],
        }

        # Set receive config for peer chain
        state_dict[peer_chain]["desired_dvn_config"][state_dict[key]["eid"]]["receive"] = {
            "requiredDVNs": [peer_dvns_by_id[id] for id in required_ids],
            "optionalDVNs": [peer_dvns_by_id[id] for id in optional_ids],
        }

# Print summary
for key in state_dict.keys():
    if key == main_chain:
        continue
    print(f"\n{key} desired configs:")
    for eid, config in state_dict[key].get("desired_dvn_config", {}).items():
        peer = eid_to_name.get(eid, eid)
        print(f"\nPeer {peer}:")
        if config["send"]["requiredDVNs"]:
            print("Send DVNs:")
            print(f"Required: {config['send']['requiredDVNs']}")
            print(f"Optional: {config['send']['optionalDVNs']}")
            print(f"Executor: {config['send']['executor']}")
        if config["receive"]["requiredDVNs"]:
            print("Receive DVNs:")
            print(f"Required: {config['receive']['requiredDVNs']}")
            print(f"Optional: {config['receive']['optionalDVNs']}")

### Set desired DVN config

In [ ]:
CONFIG_TYPE_ULN = 2
CONFIG_TYPE_READ = 1


def checksum(address):
    return Web3.to_checksum_address(address)


# Set all configs in one pass
for key in state_dict.keys():
    if key == main_chain:
        continue

    logging.info(f"\nSetting configs for {key}...")
    contract_w3 = state_dict[key]["block_relay_w3"]
    contract = state_dict[key]["block_relay"]
    oapp = checksum(contract.address)

    # Get libs
    libs = {
        "send": checksum(state_dict[key]["send_lib"])
        if state_dict[key]["send_lib"] != "unavailable"
        else None,
        "receive": checksum(state_dict[key]["receive_lib"])
        if state_dict[key]["receive_lib"] != "unavailable"
        else None,
        "read": checksum(state_dict[key]["read_lib"])
        if state_dict[key]["read_lib"] != "unavailable"
        else None,
    }

    # For each peer relationship
    for eid, config in state_dict[key]["desired_dvn_config"].items():
        is_read_channel = eid > 4294965694
        peer_name = "read_channel" if is_read_channel else eid_to_name.get(eid)
        logging.info(f"\nConfiguring {key} <-> {peer_name}")

        # Loop through send and receive configs
        for config_type in ["send", "receive"]:
            if not config[config_type]["requiredDVNs"]:
                continue

            # Skip receive for read channel
            if is_read_channel and config_type == "receive":
                continue

            # Determine correct lib and config type
            lib = libs["read"] if is_read_channel else libs[config_type]
            if not lib:
                continue

            config_type_enum = CONFIG_TYPE_READ if is_read_channel else CONFIG_TYPE_ULN

            required_dvns = sorted(
                [checksum(addr) for addr in config[config_type].get("requiredDVNs", [])]
            )
            optional_dvns = sorted(
                [checksum(addr) for addr in config[config_type].get("optionalDVNs", [])]
            )
            optional_threshold = len(optional_dvns)
            executor = config[config_type].get(
                "executor", "0x0000000000000000000000000000000000000000"
            )
            # required_dvns = []
            # optional_dvns = sorted([checksum(addr) for addr in config[config_type].get('requiredDVNs', [])])
            # optional_threshold = min(len(optional_dvns), 0)

            # full reset to defaults
            required_dvns = []
            optional_dvns = []
            optional_threshold = 0
            executor = "0x0000000000000000000000000000000000000000"

            logging.info(f"Setting {config_type} config on {key} for {peer_name}:")
            logging.info(f"- Required DVNs: {required_dvns}")
            logging.info(f"- Optional DVNs: {optional_dvns}")
            logging.info(f"- Optional threshold: {optional_threshold}")

            try:
                func = contract_w3.functions.set_lz_uln_config(
                    eid,  # peer eid
                    oapp,  # oapp
                    lib,  # lib
                    config_type_enum,  # config type
                    1,  # confirmations
                    required_dvns,  # required DVNs
                    optional_dvns,  # optional DVNs
                    optional_threshold,  # optional threshold
                    executor,
                )
                tx_hash = send_tx(state_dict[key]["w3"], func, account)
                logging.info(
                    f"Set {'read' if is_read_channel else config_type} config on {key} for {peer_name}: {tx_hash.hex()}"
                )
                time.sleep(2)
            except Exception as e:
                logging.error(f"Failed to set config on {key} for {peer_name}: {str(e)}")

####  Skipping nonce

In [ ]:
## skip nonce check
from ABIs import endpointV2_abi

chain_src = "base-sepolia"
chain_dst = "optimism-sepolia"
oapp_src = state_dict[chain_src]["block_relay"].address
oapp_dst = state_dict[chain_dst]["block_relay"].address
eid_src = state_dict[chain_src]["eid"]
eid_dst = state_dict[chain_dst]["eid"]
endpoint_w3 = state_dict[chain_dst]["w3"].eth.contract(
    address=state_dict[chain_dst]["endpoint"], abi=endpointV2_abi
)
# must encode oapp_src as bytes32 from string
oapp_src_bytes32 = Web3.to_bytes(hexstr=address.replace("0x", "").lower().rjust(64, "0"))
current_nonce = endpoint_w3.functions.inboundNonce(oapp_dst, eid_src, oapp_src_bytes32).call()
print(f"Current nonce: {current_nonce}")

# func = endpoint_w3.functions.skip(oapp_dst, eid_src, oapp_src_bytes32, current_nonce+1)
# tx_hash = send_tx(state_dict[chain_dst]["w3"], func, account)
# print(f"Sending nonce skip tx: {tx_hash.hex()}")